# Bilan hydrique à partir d'observations Météo-France horaires qualifiées

## Reconstruction des données météorologiques pour une station de référence

### Définition des paramètres

In [1]:
# Définition de la station de référence
REF_STATION_NAME = 'La Petite Claye'
REF_STATION_LATLON = [48.541356, -1.615400]
# Altitude (m)
REF_STATION_ALTITUDE = 50.

# Identification de l'API Météo-France
APPLICATION_ID = 'ZlFGb1VCNzdlQ3c5QmhSMU1IbE8xQTluOE0wYTpUS3l1YkcweGJmSTJrQlJVaGNiSkNHTXczdHNh'

# Météo-France API
METEOFRANCE_API = 'DPClim'

# Départements où chercher des stations
ID_DEPARTEMENTS = [35, 50]

# Fréquence des données climatiques
METEOFRANCE_FREQUENCE = 'horaire'

NN_NOMBRE = None
# NN_RAYON_KM = 20.
NN_RAYON_KM = 35.

# Liste des variables météorologiques utilisées pour le bilan hydrique
LISTE_VARIABLES_METEO = ['vitesse_vent_10m', 'temperature_2m', 'humidite_relative',
                         'rayonnement_global', 'precipitation']

# Fonction pour aggrégé chaque variable météorologique
VARIABLE_AGGREGATORS = {
    'vitesse_vent_10m': lambda x: x.resample('D').mean(),
    'temperature_2m': lambda x: x.resample('D').mean(),
    'humidite_relative': lambda x: x.resample('D').mean(),
    'rayonnement_global': lambda x: x.resample('D').sum(),
    'precipitation': lambda x: x.resample('D').sum(),
    'etp': lambda x: x.resample('D').sum(),
}

### Lecture de la liste des stations

In [2]:
import meteofrance

# Initialisation d'un client pour accéder à l'API Météo-France
client = meteofrance.Client(METEOFRANCE_API, application_id=APPLICATION_ID)

# Issue a series of API requests an example. For use this test, you must first subscribe to the arome api with your application
client.session.headers.update({'Accept': '*/*'})

In [3]:
import pandas as pd

LECTURE_LISTE_STATIONS = True

l_listes = []
for id_dep in ID_DEPARTEMENTS:
    filepath_liste_stations = meteofrance.get_filepath_liste_stations(
        client, frequence=METEOFRANCE_FREQUENCE, id_departement=id_dep)
    
    if LECTURE_LISTE_STATIONS:
        # Lecture de la liste des stations par département
        df_liste_stations_dep = pd.read_csv(
            filepath_liste_stations, index_col=client.id_station_label)
    else:
        # Demande de la liste des stations pour le département
        section = meteofrance.SECTION_LISTE_STATIONS
        params = {'id-departement': id_dep}
        response = meteofrance.demande(
            client, section, params=params, frequence=METEOFRANCE_FREQUENCE)
        df_liste_stations_dep = meteofrance.response_text_to_frame(
            client, response, index_col=client.id_station_label)

        # Sauvegarde de la liste des stations par département
        df_liste_stations_dep.to_csv(filepath_liste_stations)

    # Liste pour compilation
    l_listes.append(df_liste_stations_dep)

# Compilation
df_liste_stations_brute = pd.concat(l_listes, axis='index')

# Garder les stations valides seulement
df_liste_stations = meteofrance.filtrer_stations_valides(client, df_liste_stations_brute)

### Sélection des plus proches voisins de la station de référence

In [4]:
import geo

df_liste_stations_nn = geo.selection_plus_proches_voisins(
    df_liste_stations, REF_STATION_LATLON, client.latlon_labels,
    nombre=NN_NOMBRE, rayon_km=NN_RAYON_KM)

df_liste_stations_nn

,nom,typePoste,lon,lat,alt,distance
id,,,,,,
35044001,BROUALAN,2,-1.640833,48.485667,99,6
50410003,PONTORSON,1,-1.505167,48.585667,33,9
35224001,PLERGUER,2,-1.843667,48.524833,30,17
35110003,FEINS SA,1,-1.596833,48.326833,87,24
35225001,PLESDER,2,-1.924833,48.406833,56,27
35178001,MEZIERES-SUR-C.,2,-1.439000,48.308833,71,29
50531001,ST OVIN,2,-1.248667,48.682500,155,31
50218001,GRANVILLE,3,-1.613667,48.834500,37,33
35228001,DINARD,0,-2.076333,48.584833,65,34


### Obtention des données météorologiques pour les stations voisines

In [5]:
DATE_DEB_PERIODE = '2024-01-01T00:00:00Z'
DATE_FIN_PERIODE = '2024-12-31T23:00:00Z'

LECTURE_DONNEE = True

filepath_donnee = meteofrance.get_filepath_donnee_periode(
    client, DATE_DEB_PERIODE, DATE_FIN_PERIODE,
    frequence=METEOFRANCE_FREQUENCE,
    df_liste_stations=df_liste_stations_nn)
    
if LECTURE_DONNEE:
    # Lecture des données des stations pour la période
    df_meteo = pd.read_csv(
        filepath_donnee, parse_dates=[client.time_label],
        index_col=[client.id_station_donnee_label, client.time_label])
else:
    # Demande des données des stations pour la période
    variables = [client.variables_labels[METEOFRANCE_FREQUENCE][k]
                 for k in LISTE_VARIABLES_METEO]
    df_meteo = meteofrance.compiler_telechargement_des_stations_periode(
        client, df_liste_stations_nn, DATE_DEB_PERIODE, DATE_FIN_PERIODE,
        frequence=METEOFRANCE_FREQUENCE,
        read_csv_kwargs={'date_format': "%Y%m%d%H"})[variables]

    # Localisation UTC de l'indice temporel
    index = [df_meteo.index.levels[0],
             pd.DatetimeIndex(df_meteo.index.levels[1], tz='UTC')]
    df_meteo.index = df_meteo.index.set_levels(index)

    
    # Sauvegarde des données des stations pour la période par département
    df_meteo.to_csv(filepath_donnee)

### Interpolation des données météorologiques à la station de référence

Les variables sont également renommées en utilisant des noms communs à l'ensemble de ces notebooks quelque soit l'API utilisée.

In [6]:
LECTURE_DONNEE_REF_HEURE = True

str_ref_station_name = REF_STATION_NAME.lower().replace(' ', '')
filepath_donnee_ref_heure = filepath_donnee.with_name(
    filepath_donnee.stem + '_' + str_ref_station_name + filepath_donnee.suffix)

if LECTURE_DONNEE_REF_HEURE:
    # Lecture des données horaires des stations pour la période
    df_meteo_ref_heure = pd.read_csv(
        filepath_donnee_ref_heure, parse_dates=[client.time_label],
        index_col=client.time_label)
else:
    # Demande des données horaires des stations pour la période
    df_meteo_ref_heure = geo.interpolation_inverse_distance_carre(
        df_meteo, df_liste_stations_nn['distance'])
    
    # Sauvegarde des données horaires des stations pour la période
    df_meteo_ref_heure.to_csv(filepath_donnee_ref_heure)

df_meteo_ref_heure = meteofrance.renommer_variables(
    client, df_meteo_ref_heure, METEOFRANCE_FREQUENCE)

### Estimation de l'ETP journalière pour la station de référence

####  Calcul de l'ETP horaire à partir des données météorologiques

In [7]:
from etp import calcul_etp
import warnings

warnings.simplefilter('error')

# Conversion des unités
df_meteo_ref_heure.loc[:, 'temperature_2m'] = (
    df_meteo_ref_heure['temperature_2m'] + 273.15)

df_meteo_ref_heure['etp'] = calcul_etp(
    df_meteo_ref_heure, *REF_STATION_LATLON, REF_STATION_ALTITUDE)

#### Aggrégation à l'échelle journalière

In [8]:
LECTURE_DONNEE_REF = True

filepath_donnee_ref = filepath_donnee_ref_heure.with_name(
    filepath_donnee_ref_heure.name.replace(
        METEOFRANCE_FREQUENCE, 'quotidienne_estimee'))

if LECTURE_DONNEE_REF:
        # Lecture des données journalières des stations pour la période
     df_meteo_ref = pd.read_csv(
        filepath_donnee_ref, parse_dates=[client.time_label],
        index_col=client.time_label)
else:
    # Calcul des valeurs journalières des variables météo
    df_meteo_ref = pd.DataFrame(dtype=float)
    for variable, series in df_meteo_ref_heure.items():
        df_meteo_ref.loc[:, variable] = VARIABLE_AGGREGATORS[variable](
            df_meteo_ref_heure[variable])
    
    # Sauvegarde des données journalières des stations pour la période
    df_meteo_ref.to_csv(filepath_donnee_ref)

## Estimation du bilan hydrique

### Définition des paramètres

In [9]:
# RFU finale cible (mm)
RFU_CIBLE = None

# Fraction du sol occupé par des cailloux et graviers (entre 0 pour absence de cailloux et 1 pour totalité de cailloux)
FRACTION_CAILLOUX = 0.1

# Choix de la texture
TEXTURE = 'Terres limoneuses'

# Coefficient de conversion de la RU en RFU (entre 1/2 et 2/3)
RU_VERS_RFU = 2. / 3

# Fraction de la réserve utile du sol remplie d'eau (entre 0 pour une période sèche et 1 pour une période pluvieuse)
FRACTION_REMPLIE = 1.

In [10]:
# Choix de la culture
CULTURE = 'Pomme de terre'

# Choix du stade
STADE = 'Vegetation'

### Calcul du bilan hydrique

In [11]:
import bilan

df_bilan = bilan.calcul_bilan(
    TEXTURE, FRACTION_CAILLOUX,
    CULTURE, STADE,
    df_meteo_ref,
    rfu_cible=RFU_CIBLE, fraction_remplie=FRACTION_REMPLIE, ru_vers_rfu=RU_VERS_RFU)

df_bilan.describe()

,ru,profondeur_terrefine,profondeur_enracinement,rfu,rfu_cible,etp,etm_culture,precipitation,besoin_irrigation
count,3.660000e+02,366.0,366.0,3.660000e+02,3.660000e+02,366.000000,366.000000,366.000000,366.000000
mean,4.860000e+01,27.0,30.0,3.240000e+01,3.240000e+01,0.621809,0.559628,2.422033,-1.862405
std,1.423031e-14,0.0,0.0,7.115154e-15,7.115154e-15,1.177182,1.059464,4.522194,4.619568
min,4.860000e+01,27.0,30.0,3.240000e+01,3.240000e+01,0.000000,0.000000,0.000000,-38.972319
25%,4.860000e+01,27.0,30.0,3.240000e+01,3.240000e+01,0.115994,0.104394,0.058916,-2.618472
50%,4.860000e+01,27.0,30.0,3.240000e+01,3.240000e+01,0.257025,0.231322,0.418224,-0.130077
75%,4.860000e+01,27.0,30.0,3.240000e+01,3.240000e+01,0.566618,0.509956,3.189776,0.124329
max,4.860000e+01,27.0,30.0,3.240000e+01,3.240000e+01,10.507770,9.456993,40.275084,8.470470
